# Accessing ACLED Data Programmatically by Date_Range and/or Region

ACLED API documentation: https://acleddata.com/acleddatanew/wp-content/uploads/dlm_uploads/2019/01/API-User-Guide2020.pdf


#### The notebook is designed to programmatically pull ACLED data without concerns of data caps or taxing the API.

#### The API has a standard 500 data point limit but can be expanded to encompass all the data.  This notebook does this by identifying the total data points in ACLED before any queries are done using the Regions portion of the API.  If the notebook user is interested in ACLED globally the total data point count will be used as the limit parameter. This allows notebook users to only perform one pull.

#### If the notebook user is interested in a specific region it also uses the region portion of the API to pull the total data point counts for that region.  The limit parameter then becomes the total count for the specified region.  

#### The notebook then pushes it into a pandas dataframe and provides the user an option of they want to save it off as a csv.


# Info for Variables
- start_date and end_date
    - The time range you want to search on.  Formatting is YYYY-MM-DD 
- specific_reg (case sensitive)
    - Yes - if you are looking at specific region
    - No - if you aren't
- reg
    - If you selected Yes for spec_reg, this is the specific region you want to pull for. In reg variable you need to input the numeric code associated with the region below. 
        - 1 - Western Africa
        - 2 - Middle Africa
        - 3 - Eastern Africa
        - 4 - Southern Africa
        - 5 - Northern Africa
        - 7 - Southern Asia
        - 9 - South-Eastern Asia
        - 11 - Middle East
        - 12 - Europe
        - 13 - Caucasus and Central Asia
        - 14 - Central America
        - 15 - South America
        - 16 - Caribbean
- csv_create
    - Yes - if you want to save off the data
    - No - if you don't want to save the data but keeping working
- csv_file
    - entire file name making sure it ends with a .csv (ex. Tiffany_test.csv)
    
* Note - Make sure that all variables have quotes around them (example: '12')

# Populate Variables

In [11]:
##Variables
start_date = '2019-01-01'
end_date = '2019-12-31'
specific_reg = 'Yes'
reg = '14'
csv_create = 'Yes'
csv_file = 'CentralAmerica_ACLEDPROJ_Jan012019toDec012019.csv'

### Shouldn't have to change anything in the cells below

In [12]:
##import libraries and packages
import requests
import pandas as pd
from pandas.io.json import json_normalize 

In [13]:
##access regional counts
URL = "https://api.acleddata.com/region/read?terms=accept"
r = requests.get(url = URL)
data = r.json()
df_reg = json_normalize(data, 'data')

In [14]:
##get total count of events worldwide 
df_reg['count'] = df_reg['event_count'].astype(int)
total = df_reg['count'].sum(axis=0)
##create dictionary for region numeric code and counts of events in each region
dictionary_lim = pd.Series(df_reg.event_count.values, index = df_reg.region).to_dict()


In [15]:
##If doing a global query will limit to total global count
##If doing a regional query will limit to total regional count
if specific_reg == 'No':
    lim = str(total)
    print(lim)
if specific_reg == 'Yes':
    lim = dictionary_lim[reg] 
    print(lim)

21249


In [16]:
##Building of Parameters  - Should not need to change
URL = 'https://api.acleddata.com/acled/read?terms=accept'
if specific_reg == 'No':
    PARAMS = "&limit="+lim+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN"
elif specific_reg == 'Yes':
    PARAMS = "&limit="+lim+"&region="+reg+"&event_date="+start_date+"|"+end_date+"&event_date_where=BETWEEN"

In [17]:
##Request event data
r = requests.get(url = URL, params = PARAMS)
data_event = r.json()
df_event = json_normalize(data_event, 'data')

In [18]:
df_event.count()

##option to save data off as a csv or keep working
if csv_create == 'Yes':
    df_event.to_csv(csv_file)
elif csv_create == 'No':
    print('Keep on analyzing your pandas dataframe you crazy data person!')

In [19]:
df_event

,actor1,actor2,admin1,admin2,admin3,assoc_actor_1,assoc_actor_2,country,data_id,event_date,...,location,longitude,notes,region,source,source_scale,sub_event_type,time_precision,timestamp,year
0,B-18: Barrio-18,Civilians (El Salvador),La Libertad,Huizucar,,,,El Salvador,6943236,2019-12-31,...,Huizucar,-89.2383,On 31th December 2019 in Huizucar (La Libertad...,Central America,El Salvador.com,National,Attack,1,1582839958,2019
1,Unidentified Gang (Mexico),Civilians (Mexico),Michoacan,Cuitzeo,,,,Mexico,6958095,2019-12-31,...,Cuitzeo del Porvenir,-101.1396,"On 31 December 2019, in Chupicuaro community, ...",Central America,La Voz de Michoacan,Subnational,Attack,2,1582840326,2019
2,Unidentified Gang (Mexico),Civilians (Mexico),Morelos,Amacuzac,,,Women (Mexico); Government of Mexico (2018-),Mexico,6926873,2019-12-31,...,Amacuzac,-99.3695,"On 31 December 2019, in Amacuzac, Morelos, a w...",Central America,El Sol de Acapulco; La Verdad,National,Attack,2,1582839780,2019
3,Unidentified Armed Group (Mexico),Civilians (Mexico),Veracruz,Veracruz,,,,Mexico,6926874,2019-12-31,...,Veracruz,-96.1530,"On 31 December 2018, in the state of Veracruz,...",Central America,E-Consulta; Sintesis,National,Abduction/forced disappearance,1,1582839780,2019
4,El Pelon de las Playas Cartel,Civilians (Mexico),Tabasco,Centro,,,,Mexico,6926875,2019-12-31,...,Villahermosa,-92.9194,"On 31 December 2019, in Villahermosa, Tabasco,...",Central America,Tabasco Hoy; El Norte; La Verdad,Subnational-National,Looting/property destruction,1,1582839780,2019
5,Protesters (Mexico),,Veracruz,Xalapa,,,,Mexico,6926876,2019-12-31,...,Xalapa-Enriquez,-96.9226,"On 31 December 2019, in Xalapa, Veracruz, a gr...",Central America,Imagen del Golfo; El Proceso,National,Peaceful protest,1,1582839780,2019
6,Unidentified Armed Group (Mexico),Civilians (Mexico),Chihuahua,Juarez,,,,Mexico,6957604,2019-12-31,...,Ciudad Juarez,-106.4850,"On 31 December 2019, Colonia Felipe Angeles, C...",Central America,La Verdad,National,Attack,2,1582840325,2019
7,Unidentified Armed Group (Mexico),Civilians (Mexico),Hidalgo,Nopala de Villagran,,,,Mexico,6956847,2019-12-31,...,Nopala de Villagran,-99.6454,"On 31 December 2019, in Nopala de Villagran, H...",Central America,La Silla Rota,National,Attack,2,1582840323,2019
8,Rioters (Mexico),Police Forces of Mexico (2018-),Puebla,Puebla,,Labour Group (Mexico),,Mexico,6968624,2019-12-31,...,Heroica Puebla de Zaragoza,-98.1975,"On 31 December 2019, in Zapata Market in Puebl...",Central America,Sintesis,National,Mob violence,1,1582840374,2019
9,MS-13: Mara Salvatrucha,Police Forces of El Salvador (2019-),La Union,La Union,,,,El Salvador,7045709,2019-12-31,...,La Union,-87.8439,As reported on 31th December 2019 in La Union ...,Central America,El Mundo (El Salvador),National,Armed clash,2,1586300978,2019


In [20]:
print(len(df_event.columns))

31
